In [18]:
import numpy as np
from scipy.io import loadmat

data = loadmat('//home//khalid//pythonfiles//classExo3.mat')

#print(data)
print('X shape = ', data['X'].shape)
print(data['X'])

print('\nY shape = ', data['y'].shape)
print(data['y'])

X shape =  (5000, 400)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

Y shape =  (5000, 1)
[[10]
 [10]
 [10]
 ...
 [ 9]
 [ 9]
 [ 9]]


In [19]:
#functions
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def cost(theta, X, y, learningRate):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    
    first = np.multiply(-y, np.log(sigmoid(X * theta.T)))
    second = np.multiply((1-y), np.log(1-sigmoid(X * theta.T)))
    
    reg = (learningRate / 2 * len(X)) * np.sum(np.power(theta[:,1:theta.shape[1]], 2))
    
    return np.sum(first - second) / (len(X)) + reg

def gradient_with_loop(theta, X, y, learningRate):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    
    parameters = int(theta.ravel().shape[1])
    grad = np.zeros(parameters)
    
    error = sigmoid(X * theta.T) - y
    
    for i in range(parameters):
        term = np.multiply(error, X[:,i])
        
        if(i==0):
            grad[i] = np.sum(term)/len(X)
        else:
            grad[i] = (np.sum(term) / len(X)) + ((learningRate / len(X)) * theta[:,i])
    
    return grad

def gradient(theta, X, y, learningRate):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    
    parameters = int(theta.ravel().shape[1])
    error = sigmoid(X * theta.T) - y
    
    grad = ((X.T * error) / len(X)).T + ((learningRate / len(X)) * theta)
    
    # intercept gradient is not regularized
    grad[0, 0] = np.sum(np.multiply(error, X[:,0])) / len(X)
    
    return np.array(grad).ravel()

from scipy.optimize import minimize

def one_vs_all(X, y, num_labels, learning_rate):
    rows = X.shape[0] #5000
    params = X.shape[1] #400
    
    # k X (n + 1) array for the parameters of each of the k classifiers
    all_theta = np.zeros((num_labels, params + 1))
    
    print('all_theta shape ' , all_theta.shape)
    # insert a column of ones at the beginning for the intercept term
    X = np.insert(X, 0, values=np.ones(rows), axis=1)
    print('X shape ' , X.shape)
    
    #classification code
    # labels are 1-indexed instead of 0-indexed
    for i in range(1, num_labels + 1):
        theta = np.zeros(params + 1)
        y_i = np.array([1 if label == i else 0 for label in y])
        y_i = np.reshape(y_i, (rows, 1))
        # minimize the objective function
        fmin = minimize(fun=cost, x0=theta, args=(X, y_i, learning_rate), method='TNC', jac=gradient)
        all_theta[i-1,:] = fmin.x
        
    return all_theta
            

In [29]:
rows = data['X'].shape[0] #5000
params = data['X'].shape[1] # 400

print('rows = ' ,rows)
print('params = ' , params)

rows =  5000
params =  400


In [30]:
all_theta = np.zeros((10, params + 1))

print('all_theta \n' , all_theta)
print('all_theta shape \n' , all_theta.shape)

all_theta 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
all_theta shape 
 (10, 401)


In [31]:
X = np.insert(data['X'], 0, values=np.ones(rows), axis=1)

# print(X)
print('X Shape = ' , X.shape)

X Shape =  (5000, 401)


In [33]:
theta = np.zeros(params + 1)
#print('theta \n' , theta )

In [24]:
y_0 = np.array([1 if label == 0 else 0 for label in data['y']])
'''
explinaton
4   0
5   0
8   0
0   1
2   0
10  0
0   1
4   0
0   1
0   1
7   0
9   0
'''

# print('y_0')
# print(y_0.shape)
# print(y_0) #print(y_0[0:99])

'\nexplinaton\n4   0\n5   0\n8   0\n0   1\n2   0\n10  0\n0   1\n4   0\n0   1\n0   1\n7   0\n9   0\n'

In [25]:
y_0 = np.reshape(y_0, (rows, 1)) #T

# print('y_0')
# print(y_0.shape)
# print(y_0)

In [26]:
print()
print('X.shape = ',X.shape)
print()
print('y.shape = ',y_0.shape)
print()
print('theta.shape = ',theta.shape)
print()
print('all_theta.shape = ',all_theta.shape)

print()
print('data array = ' , np.unique(data['y']))


X.shape =  (5000, 401)

y.shape =  (5000, 1)

theta.shape =  (401,)

all_theta.shape =  (10, 401)

data array =  [ 1  2  3  4  5  6  7  8  9 10]


In [27]:
print()
all_theta = one_vs_all(data['X'], data['y'], 1, 1) #first 1, let us decide how many m we want classifed

print('Theta shape = ' , all_theta.shape)
# print('Theta = ')
# print(all_theta)


all_theta shape  (1, 401)
X shape  (5000, 401)
Theta shape =  (1, 401)


In [28]:
def predict_all(X, all_theta):
    rows = X.shape[0]
    params = X.shape[1]
    num_labels = all_theta.shape[0]
    
    # same as before, insert ones to match the shape
    X = np.insert(X, 0, values=np.ones(rows), axis=1)
    
    # convert to matrices
    X = np.matrix(X)
    all_theta = np.matrix(all_theta)
    
    # compute the class probability for each class on each training instance
    h = sigmoid(X * all_theta.T)
    
    # create array of the index with the maximum probability
    h_argmax = np.argmax(h, axis=1)
    
    # because our array was zero-indexed we need to add one for the true label prediction
    h_argmax = h_argmax + 1
    
    return h_argmax



y_pred = predict_all(data['X'], all_theta)
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, data['y'])]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print ('accuracy = {0}%'.format(accuracy * 100))

accuracy = 10.0%
